# RNN 기본예제

* 일부 코드 [김성훈 교수님 TensorFlow 강의자료](https://github.com/hunkim/DeepLearningZeroToAll) 참고

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

import tensorflow as tf

slim = tf.contrib.slim
rnn = tf.contrib.rnn

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))

In [ ]:
# One hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

![image](https://cloud.githubusercontent.com/assets/901975/23348727/cc981856-fce7-11e6-83ea-4b187473466b.png)

## Basic RNN Cell

In [ ]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('one_cell', reuse=tf.AUTO_REUSE) as scope:
    # One cell RNN input_dim (4) -> output_dim (2)
    hidden_size = 2
    cell = rnn.BasicRNNCell(num_units=hidden_size)
    print("output size: {}, state size: {}".format(cell.output_size, cell.state_size))

    x_data = np.array([[h]], dtype=np.float32) # x_data = [[[1,0,0,0]]]
    print("x_data: {}".format(x_data))
    outputs, state = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, state_ = sess.run([outputs, state])
    print("outputs: {}".format(outputs_))
    print("state: {}".format(state_))

![image](https://cloud.githubusercontent.com/assets/901975/23383634/649efd0a-fd82-11e6-925d-8041242743b0.png)

In [ ]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('add_sequances', reuse=tf.AUTO_REUSE) as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = rnn.BasicRNNCell(num_units=hidden_size)

    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print("x_data shape: {}".format(x_data.shape))
    print("x_data: {}".format(x_data))
    outputs, state = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, state_ = sess.run([outputs, state])
    print("outputs: \n{}".format(outputs_))
    print("state: {}".format(state_))

## [`tf.nn.static_rnn`](https://www.tensorflow.org/api_docs/python/tf/nn/static_rnn)

* `tf.nn.static_rnn` low level code

```python
state = cell.zero_state(...)
outputs = []
for input_ in inputs:
  output, state = cell(input_, state)
  outputs.append(output)
return (outputs, state)
```

![image](https://cloud.githubusercontent.com/assets/901975/23383681/9943a9fc-fd82-11e6-8121-bd187994e249.png)

## Basic LSTM Cell

In [ ]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('3_batches_LSTM', reuse=tf.AUTO_REUSE) as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)

    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, state = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, state_ = sess.run([outputs, state])
    print("outputs: \n{}\n".format(outputs_))
    #print("memory cell state: \n{}".format(state_[0])) # print memory cell
    #print("hidden cell state: \n{}".format(state_[1])) # print hidden state
    print("memory cell state: \n{}".format(state_.c)) # print memory cell
    print("hidden cell state: \n{}".format(state_.h)) # print hidden state

## GRU Cell

In [ ]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('3_batches_GRU', reuse=tf.AUTO_REUSE) as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)

    hidden_size = 2
    cell = rnn.GRUCell(num_units=hidden_size)
    outputs, state = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, state_ = sess.run([outputs, state])
    print("outputs: \n{}\n".format(outputs_))
    print("hidden cell state: \n{}".format(state_)) # print hidden state

### Dynamic length

In [ ]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('3_batches_dynamic_length', reuse=tf.AUTO_REUSE) as scope:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)

    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, state = tf.nn.dynamic_rnn(cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, state_ = sess.run([outputs, state])
    print("outputs: \n{}\n".format(outputs_))
    print("memory cell state: \n{}".format(state_.c)) # print memory cell
    print("hidden cell state: \n{}".format(state_.h)) # print hidden state

### Feed initial state

In [ ]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('initial_state', reuse=tf.AUTO_REUSE) as scope:
    batch_size = 3
    x_data = np.array([[h, e, l, l, o],
                      [e, o, l, l, l],
                      [l, l, e, e, l]], dtype=np.float32)

    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch: 3
    hidden_size=2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, state = tf.nn.dynamic_rnn(cell, x_data,
                                       initial_state=initial_state,
                                       dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, state_ = sess.run([outputs, state])
    print("outputs: \n{}\n".format(outputs_))
    print("memory cell state: \n{}".format(state_.c)) # print memory cell
    print("hidden cell state: \n{}".format(state_.h)) # print hidden state

## Create new input data

In [ ]:
# Create new input data
batch_size=3
sequence_length=5
input_dim=3

x_data = np.arange(45, dtype=np.float32).reshape(batch_size, sequence_length, input_dim)
print(x_data)  # [batch, sequence_length, input_dim]

## Bi-directional RNN

![bi-directional rnn](../figures/RNN-bidirectional.png)

### Bi-directional RNN with basic RNN Cell

In [ ]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('bi-directional_RNN', reuse=tf.AUTO_REUSE) as scope:
    # bi-directional rnn
    cell_fw = rnn.BasicRNNCell(num_units=4)
    cell_bw = rnn.BasicRNNCell(num_units=4)

    # x_data.shape = (3, 5, 3) [batch, sequence_length, input_dim]
    # outputs.shape = two element tuple of (3, 5, 4) [batch, sequence_length, input_dim] shape
        # outputs[0]: cell_fw, outputs[1]: cell_bw
    # state.shape = two element tuple (3, 4) [batch, sequence_length, input_dim]
        # states[0]: cell_fw, states[1]: cell_bw
    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                      sequence_length=[2, 3, 1],
                                                      dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, states_ = sess.run([outputs, states])
    print("foward outputs: shape {}\n{}\n".format(outputs_[0].shape, outputs_[0]))
    print("forward hidden cell state: \n{}\n".format(states_[0]))
    print("backward outputs: shape {}\n{}\n".format(outputs_[1].shape, outputs_[1]))
    print("backward hidden cell state: \n{}".format(states_[1]))

### Bi-directional RNN with basic LSTM Cell

In [ ]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('bi-directional_LSTM', reuse=tf.AUTO_REUSE) as scope:
    # bi-directional rnn
    cell_fw = rnn.BasicLSTMCell(num_units=4, state_is_tuple=True)
    cell_bw = rnn.BasicLSTMCell(num_units=4, state_is_tuple=True)

    # x_data.shape = (3, 5, 3) [batch, sequence_length, input_dim]
    # outputs.shape = two element tuple of (3, 5, 4) [batch, sequence_length, input_dim] shape
        # outputs[0]: cell_fw, outputs[1]: cell_bw
    # state.shape = two element tuple (3, 4) [batch, sequence_length, input_dim]
        # states[0]: cell_fw, states[1]: cell_bw
    outputs, state = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                      sequence_length=[2, 3, 1],
                                                      dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, state_ = sess.run([outputs, state])
    print("foward outputs: shape: {}\n{}\n".format(outputs_[0].shape, outputs_[0]))
    print("forward memory cell state: \n{}".format(state_[0].c))
    print("forward hidden cell state: \n{}\n".format(state_[0].h))
    print("backward outputs: shape: {}\n{}\n".format(outputs_[1].shape, outputs_[1]))
    print("backward memory cell state: \n{}".format(state_[1].c))
    print("backward hidden cell state: \n{}".format(state_[1].h))

## Multi-Layer RNN

![Multi-layer rnn](../figures/Multi-layer_rnn.png)

### Multi-Layer RNN with basic RNN Cell

In [ ]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('MultiRNN_RNN', reuse=tf.AUTO_REUSE) as scope:
    # MultiLayer RNN
    num_layers = 3
    multi_cells = rnn.MultiRNNCell([rnn.BasicRNNCell(4) for _ in range(num_layers)])

    outputs, states = tf.nn.dynamic_rnn(multi_cells, x_data,
                                        sequence_length=[2, 3, 1],
                                        dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, states_ = sess.run([outputs, states])
    print("outputs: shape: {}\n{}\n".format(outputs_.shape, outputs_))
    print("Number of Layers: {}".format(len(states_))) 
    for i in range(num_layers):
      print("Layer {} hidden cell state: \n{}\n".format(i+1, states_[i]))

### Multi-Layer RNN with basic LSTM Cell

In [ ]:
with tf.Session(config=sess_config) as sess:
  with tf.variable_scope('MultiRNN_LSTM', reuse=tf.AUTO_REUSE) as scope:
    # MultiLayer RNN
    def lstm_cell(hidden_size):
      cell = rnn.BasicLSTMCell(
          num_units=hidden_size, state_is_tuple=True)
      return cell

    num_layers = 3
    multi_cells = rnn.MultiRNNCell([lstm_cell(4) for _ in range(num_layers)],
                                   state_is_tuple=True)
    outputs, states = tf.nn.dynamic_rnn(multi_cells, x_data,
                                        sequence_length=[2, 3, 1],
                                        dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    outputs_, states_ = sess.run([outputs, states])
    print("outputs: shape: {}\n{}\n".format(outputs_.shape, outputs_))
    print("Number of Layers: {}".format(len(states_))) 
    for i in range(num_layers):
      print("Layer {} memory cell state: \n{}\n".format(i+1, states_[i].c))
      print("Layer {} hidden cell state: \n{}\n".format(i+1, states_[i].h))